In [ ]:
import os, re
import urllib.request, json 

import tqdm
import time

import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

import optuna
import pickle as pkl

from joblib import Parallel, delayed

with open('./notebook_artifacts/usda_nass_api_key.txt') as f:
    api_key = f.readlines()
    api_key = api_key[0]
    
cache_path = './notebook_artifacts/9_historical/'    

save_path = './data/Processed/'
meta = pd.read_csv(save_path+"meta0.csv")
phno = pd.read_csv(save_path+"phno3.csv")

# Add State and County data into 2022

In [ ]:
# drop sites in canada
canada_list = [
    'ONH2_2019', 'ONH2_2017', 'ONH2_2014', 'ONH2_2016', 'ONH2_2015', 
    'ONH1_2017', 'ONH1_2015', 'ONH1_2014', 'ONH1_2016']
mask = [False if e in canada_list else True for e in phno.Env]
phno = phno.loc[mask, ].reset_index().drop(columns = ['index'])


mask = [False if e in canada_list else True for e in meta.Env]
meta = meta.loc[mask, ].reset_index().drop(columns = ['index'])


In [ ]:
# phno
latlons = meta.loc[(meta.Year == 2022), ['Env', 'Latitude_of_Field', 'Longitude_of_Field']].drop_duplicates().reset_index().drop(columns = 'index')
latlons2 =meta.loc[(meta.Year != 2022), ['Env', 'Latitude_of_Field', 'Longitude_of_Field']].drop_duplicates().reset_index().drop(columns = 'index')

In [ ]:
# initially only looked at counties in the test set
latlons['county'] = ''
latlons['zip'] = ''
latlons['states'] = ''

zipcodes = [
    '52249',  '50014',  '68507', '53593', '53593', '47906', '19947', '52621', 
    '51443', '61820', '13026', '13026', '48864', '56093', '30677', '65203',  
    '45368', '69101', '69101', '54930', '29670', '77879', '77879', '77879', 
    '31793', '27849']
counties = [
    'Benton', 'Story', 'Lancaster','Dane','Dane','Tippecanoe','Sussex', 
    'Washington', 'Carroll', # originally typed as carrol 
    'Champaign', 'Cayuga', 'Cayuga', 'Ingham', 
    'Waseca', 'Oconee', 'Boone', 'Clark', 'Lincoln', 'Lincoln', 'Waushara', 
    'Anderson', 'Burleson', 'Burleson', 'Burleson', 'Tift', 'Bertie']
states =   [
    'IA', 'IA', 'NE', 'WI', 'WI', 'IN', 'DE', 'IA', 'IA', 'IL', 'NY', 'NY', 
    'MI', 'MN', 'GA', 'MO', 'OH', 'NE', 'NE', 'WI', 'SC', 'TX', 'TX', 'TX', 
    'GA', 'NC']

for i in range(len(zipcodes)):
    latlons.loc[i, 'zip'] = zipcodes[i]    
for i in range(len(counties)):
    latlons.loc[i, 'county'] = counties[i]
for i in range(len(states)):
    latlons.loc[i, 'states'] = states[i]

# latlons#.loc[(latlons.zip == ''), ]

In [ ]:
latlons.loc[latlons.states == 'IA']

,Env,Latitude_of_Field,Longitude_of_Field,county,zip,states
0,IAH3_2022,41.987866,-92.258049,Benton,52249,IA
1,IAH4_2022,41.994648,-93.690428,Story,50014,IA
7,IAH1_2022,41.197664,-91.487201,Washington,52621,IA
8,IAH2_2022,42.067492,-94.729009,Carroll,51443,IA


In [ ]:
# what about other counties?
threshold = 0.05 # for euclid dist between existing/non latlon
threshold = 0.1 # for euclid dist between existing/non latlon
tmp = latlons.merge(latlons2, how = 'outer')


## Add labels manually
# https://gps-coordinates.org/what-county-am-i-in.php
add_data =  [
    #Lat         Lon       county  state
    [34.727251, -90.760326, 'Lee', 'AR'],
    [35.296577, -77.566463 , 'Lenoir', 'NC'],
    [35.483467,-77.569042, 'Greene', 'NC'],
    [35.666783, -78.492372, 'Johnston', 'NC'],
    [35.838614, -90.665302, 'Craighead', 'AR'],
    [37.810107, -100.775188, 'Finney', 'KS'],
    [38.892640, -92.205753, 'Boone', 'MO'],
    [38.913500, -92.281389, 'Boone', 'MO'],
    [39.144807, -96.628982, 'Riley', 'KS'],
    [39.216418,-96.604822, 'Riley', 'KS'],
    [40.648500,-104.999782, 'Larimer', 'CO'],
    [41.158245,-101.987296, 'Keith', 'NE'],
    [41.162000, -96.409000, 'Saunders', 'NE'],
    [41.290979,-96.416643, 'Saunders', 'NE'],
    [41.380423, -96.559884, 'Saunders', 'NE'],
    [42.031938, -93.793332, 'Boone', 'IA'],
    [42.412067,-84.295262, 'Ingham', 'MI'],
#     [42.452213	-81.881844, '', ''], # Canada
#     [43.495460,-80.425969, '', ''],
#     [, '', ''],
#     [, '', ''],
#     [, '', ''],
#     [, '', ''],
    [44.208745, -102.928745, 'Pennington', 'SD']
]
search_tol = 0.01

for i in range(len(add_data)):
    mask = ( (tmp.county.isna()
        ) & ((tmp.Latitude_of_Field  - add_data[i][0]) < search_tol
        ) & ((tmp.Longitude_of_Field - add_data[i][1]) < search_tol))

    tmp.loc[mask, 'county'] = add_data[i][2]
    tmp.loc[mask, 'states'] = add_data[i][3]
    
## Add labels by distance
tmp_has_county = tmp.loc[(tmp.county.notna()), :]
tmp = tmp.loc[(tmp.county.isna()), :]

tmp = tmp.loc[:, ['Env', 'Latitude_of_Field', 'Longitude_of_Field', 'county', 'states']
       ].drop_duplicates(
       ).sort_values(['Latitude_of_Field', 'Longitude_of_Field']).reset_index().drop(columns = ['index'])

for i in tmp.index:
    tmp_has_county['Latitude_Diff'] = np.nan
    tmp_has_county['Longitude_Diff'] = np.nan
    tmp_has_county['Euclid_Diff'] = np.nan

    tmp_has_county['Latitude_Diff']  = (tmp_has_county.loc[:, 'Latitude_of_Field']  - tmp.loc[i, 'Latitude_of_Field'])
    tmp_has_county['Longitude_Diff'] = (tmp_has_county.loc[:, 'Longitude_of_Field'] - tmp.loc[i, 'Longitude_of_Field'])
    tmp_has_county['Euclid_Diff']    = np.sqrt(tmp_has_county['Latitude_Diff']**2 + tmp_has_county['Longitude_Diff']**2)

    tmp_has_county.sort_values(['Euclid_Diff'])

    min_diff = min(tmp_has_county['Euclid_Diff'])
    mask = (tmp_has_county['Euclid_Diff'] == min_diff)

    if min_diff > threshold:
        pass
    else:
        tmp.loc[i, 'county'] = list(tmp_has_county.loc[mask, 'county'])[0]
        tmp.loc[i, 'states'] = list(tmp_has_county.loc[mask, 'states'])[0]

In [ ]:
print(tmp.loc[tmp.county.isna(), ].shape[0])
tmp.loc[tmp.county.isna(), ]

0


,Env,Latitude_of_Field,Longitude_of_Field,county,states


In [ ]:
tmp = tmp.loc[tmp.county.notna(), ].copy() # gets rid of ONH\d values

In [ ]:
cols = ['Env', 'Latitude_of_Field', 'Longitude_of_Field', 'county', 'states']

latlons = tmp.loc[:, cols
            ].merge(tmp_has_county.loc[:, cols], how = 'outer'
            ).drop(columns = ['Latitude_of_Field', 'Longitude_of_Field']
            ).drop_duplicates()


latlons.loc[:, 'county'] = [e.upper() for e in latlons['county']]

In [ ]:
dat = phno.merge(latlons, how = 'outer')
dat = dat.rename(columns = {
    'county':'County', 
    'states':'State'})
dat

,Env,Year,Hybrid,Yield_Mg_ha,County,State
0,DEH1_2014,2014,M0088/LH185,5.721725,SUSSEX,DE
1,DEH1_2014,2014,M0143/LH185,11.338246,SUSSEX,DE
2,DEH1_2014,2014,M0003/LH185,6.540810,SUSSEX,DE
3,DEH1_2014,2014,M0035/LH185,10.366857,SUSSEX,DE
4,DEH1_2014,2014,M0052/LH185,10.908814,SUSSEX,DE
...,...,...,...,...,...,...
134179,WIH3_2022,2022,W10010_0337/LH244,NaN,WAUSHARA,WI
134180,WIH3_2022,2022,W10010_0346/LH244,NaN,WAUSHARA,WI
134181,WIH3_2022,2022,W10010_0358/LH244,NaN,WAUSHARA,WI
134182,WIH3_2022,2022,W10010_0381/LH244,NaN,WAUSHARA,WI


# Process NASS data

In [ ]:
# Building up a dataset from historical entries
# Commodity: CORN
# Category: YIELD
# Geographic Level: County
# State: #select a set of states that is small enough to not hit the 50000 entry download limit
# e.g. './notebook_artifacts/9_historical/5AD68EDD-41CB-308E-B5F0-530F45F54F3C.csv'

# There's no 2022 data included in the downloaded data so I'll use the state level too
nass_state_csv = 'E33B3659-1330-3D9B-BFC3-DF934E2AEE99.csv'

nass_csvs = [e for e in os.listdir(cache_path) if re.match('[A-Z0-9]{8}\-[A-Z0-9]{4}\-[A-Z0-9]{4}\-[A-Z0-9]{4}\-[A-Z0-9]{12}\.csv', e)]
nass_csvs = [e for e in nass_csvs if e != nass_state_csv]

nass_df = pd.concat([pd.read_csv(cache_path+nass_csv) for nass_csv in nass_csvs])

In [ ]:
nass_df = nass_df.loc[:, [
#     'Program',
    'Year',
#     'Period',
#     'Week Ending',
#     'Geo Level',
    'State',
#     'State ANSI',
    'Ag District',
#     'Ag District Code',
    'County',
#     'County ANSI',
#     'Zip Code',
#     'Region',
#     'watershed_code',
#     'Watershed',
#     'Commodity',
    'Data Item',
#     'Domain',
#     'Domain Category',
    'Value',
#     'CV (%)'
]]

[[len(set(nass_df[e])), e] for e in list(nass_df)]

[[112, 'Year'],
 [41, 'State'],
 [83, 'Ag District'],
 [1650, 'County'],
 [9, 'Data Item'],
 [2364, 'Value']]

In [ ]:
data_dict = {
                "CORN, GRAIN, IRRIGATED - YIELD, MEASURED IN BU / ACRE": "GRN_IRR_BUpACRE",
               "CORN, GRAIN - YIELD, MEASURED IN BU / NET PLANTED ACRE": "GRN_BUpNETPLANTEDACRE",
"CORN, GRAIN, NON-IRRIGATED - YIELD, MEASURED IN BU / NET PLANTED ACRE": "GRN_NON_IRR_BUpNETPLANTEDACRE",
             "CORN, SILAGE, IRRIGATED - YIELD, MEASURED IN TONS / ACRE": "SLG_IRR_TONSpACRE",
         "CORN, SILAGE, NON-IRRIGATED - YIELD, MEASURED IN TONS / ACRE": "SLG_NON_IRR_TONSpACRE",
            "CORN, GRAIN, NON-IRRIGATED - YIELD, MEASURED IN BU / ACRE": "GRN_NON_IRR_BUpACRE",
    "CORN, GRAIN, IRRIGATED - YIELD, MEASURED IN BU / NET PLANTED ACRE": "GRN_IRR_BUpNETPLANTEDACRE",
                        "CORN, SILAGE - YIELD, MEASURED IN TONS / ACRE": "SLG_TONSpACRE",
                           "CORN, GRAIN - YIELD, MEASURED IN BU / ACRE": "GRN_BUpACRE"
}

In [ ]:
nass_df = nass_df.merge(pd.DataFrame(data_dict, index = [0]).T.reset_index(
                ).rename(columns = {'index':'Data Item', 0:'Key'})
                ).drop(columns = ['Data Item'])
nass_df = nass_df.rename(columns = {'Ag District': 'AgDistrict'})

In [ ]:
nass_df

,Year,State,AgDistrict,County,Value,Key
0,2021,NEBRASKA,NaN,OTHER COUNTIES,201.6,GRN_BUpACRE
1,2021,NEBRASKA,CENTRAL,GREELEY,222.6,GRN_BUpACRE
2,2021,NEBRASKA,CENTRAL,HOWARD,198.6,GRN_BUpACRE
3,2021,NEBRASKA,CENTRAL,SHERMAN,222.0,GRN_BUpACRE
4,2021,NEBRASKA,CENTRAL,VALLEY,229.9,GRN_BUpACRE
...,...,...,...,...,...,...
305470,1975,COLORADO,NORTHEAST,MORGAN,15.5,GRN_NON_IRR_BUpNETPLANTEDACRE
305471,1975,COLORADO,NORTHEAST,SEDGWICK,17.0,GRN_NON_IRR_BUpNETPLANTEDACRE
305472,1975,COLORADO,NORTHEAST,WELD,16.0,GRN_NON_IRR_BUpNETPLANTEDACRE
305473,1975,COLORADO,SOUTHEAST,BACA,11.5,GRN_NON_IRR_BUpNETPLANTEDACRE


In [ ]:
nass_df_wide = nass_df.pivot(columns='Key', 
                             values='Value', 
                             index=['Year', 'State', 'AgDistrict', 'County']
                            ).reset_index()
nass_df_wide

Key,Year,State,AgDistrict,County,GRN_BUpACRE,GRN_BUpNETPLANTEDACRE,GRN_IRR_BUpACRE,GRN_IRR_BUpNETPLANTEDACRE,GRN_NON_IRR_BUpACRE,GRN_NON_IRR_BUpNETPLANTEDACRE,SLG_IRR_TONSpACRE,SLG_NON_IRR_TONSpACRE,SLG_TONSpACRE
0,1910,NEBRASKA,CENTRAL,BUFFALO,24.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1910,NEBRASKA,CENTRAL,CUSTER,21.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1910,NEBRASKA,CENTRAL,DAWSON,17.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1910,NEBRASKA,CENTRAL,GREELEY,24.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1910,NEBRASKA,CENTRAL,HALL,31.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
191333,2021,WISCONSIN,WEST CENTRAL,MONROE,146.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.0
191334,2021,WISCONSIN,WEST CENTRAL,PEPIN,176.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
191335,2021,WISCONSIN,WEST CENTRAL,PIERCE,185.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.5
191336,2021,WISCONSIN,WEST CENTRAL,ST CROIX,171.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.5


In [ ]:
# drop cols with low fill rate
nass_df_wide = nass_df_wide.drop(columns = [
    'GRN_BUpNETPLANTEDACRE',
    'GRN_IRR_BUpNETPLANTEDACRE',
    'GRN_NON_IRR_BUpNETPLANTEDACRE',
    'SLG_IRR_TONSpACRE',
    'SLG_NON_IRR_TONSpACRE',
])

In [ ]:
# nass_df_wide.loc[nass_df_wide.Year == 2022, ]

# add a placeholder for 2022 data

tmp = nass_df_wide.loc[:, ['State', 'AgDistrict', 'County']].drop_duplicates().reset_index().drop(columns = ['index'])
tmp['Year'] = 2022

nass_df_wide = nass_df_wide.merge(tmp, how = 'outer')


In [ ]:
nass_df_wide.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 194572 entries, 0 to 194571
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Year                 194572 non-null  int64  
 1   State                194572 non-null  object 
 2   AgDistrict           194475 non-null  object 
 3   County               194572 non-null  object 
 4   GRN_BUpACRE          185869 non-null  float64
 5   GRN_IRR_BUpACRE      16625 non-null   float64
 6   GRN_NON_IRR_BUpACRE  14742 non-null   float64
 7   SLG_TONSpACRE        49158 non-null   float64
dtypes: float64(4), int64(1), object(3)
memory usage: 13.4+ MB


In [ ]:
nass_state_df = pd.read_csv(cache_path+nass_state_csv)

nass_state_df = nass_state_df.loc[:, 
[
#'Program',
 'Year',
#  'Period',
#  'Week Ending',
#  'Geo Level',
 'State',
#  'State ANSI',
#  'Ag District',
#  'Ag District Code',
#  'County',
#  'County ANSI',
#  'Zip Code',
#  'Region',
#  'watershed_code',
#  'Watershed',
#  'Commodity',
 'Data Item',
#  'Domain',
#  'Domain Category', #  going to ignore this, 'IRRIGATION METHOD, PRIMARY: (GRAVITY)',
                                            #  'IRRIGATION METHOD, PRIMARY: (PRESSURE)',
                                            #  'IRRIGATION STATUS: (ANY ON OPERATION)',
                                            #  'NOT SPECIFIED'
 'Value',
#  'CV (%)'
]].drop_duplicates()

# .pivot(columns='Key', 
#                              values='Value', 
#                              index=['Year', 'State', 'AgDistrict', 'County']
#                             ).reset_index(

In [ ]:
 nass_state_df = nass_state_df.loc[(nass_state_df.Value != ' (D)'), ].copy()

In [ ]:
nass_state_df.loc[:, ['Value']] = nass_state_df.loc[:, ['Value']].astype(float)

/tmp/ipykernel_2025081/1692625117.py:1: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  nass_state_df.loc[:, ['Value']] = nass_state_df.loc[:, ['Value']].astype(float)


In [ ]:
nass_state_df = nass_state_df.merge(pd.DataFrame(data_dict, index = [0]).T.reset_index(
                ).rename(columns = {'index':'Data Item', 0:'Key'})
                ).drop(columns = ['Data Item']
                ).groupby(['Year', 'State', 'Key']
                ).agg(Value = ('Value', np.nanmean)
                ).reset_index()

In [ ]:
# nass_df_wide = nass_df
nass_state_df = nass_state_df.pivot(
        columns='Key', 
        values='Value', 
        index=['Year', 'State']
    ).reset_index().loc[:, [
        'Year', 'State', 'GRN_BUpACRE', 'SLG_TONSpACRE']
    ].rename(columns = {
    'GRN_BUpACRE': 'STATE_GRN_BUpACRE', 
    'SLG_TONSpACRE': 'STATE_SLG_TONSpACRE'})

nass_state_df

Key,Year,State,STATE_GRN_BUpACRE,STATE_SLG_TONSpACRE
0,1866,ALABAMA,9.000000,NaN
1,1866,ARKANSAS,18.000000,NaN
2,1866,CALIFORNIA,28.000000,NaN
3,1866,CONNECTICUT,34.000000,NaN
4,1866,DELAWARE,23.000000,NaN
...,...,...,...,...
7255,2022,TENNESSEE,127.666667,NaN
7256,2022,TEXAS,104.500000,NaN
7257,2022,VIRGINIA,161.750000,NaN
7258,2022,WASHINGTON,242.500000,NaN


In [ ]:
# impute missing values
nass_df_big = nass_df_wide.merge(nass_state_df).drop(columns = [
    'GRN_IRR_BUpACRE', 'GRN_NON_IRR_BUpACRE', 'SLG_TONSpACRE']) # drop low completion (~1/10) cols

nass_df_big = nass_df_big.loc[~(nass_df_big.AgDistrict.isna()), ] # drop "OTHER COUNTIES"

In [ ]:
# https://www.faa.gov/air_traffic/publications/atpubs/cnt_html/appendix_a.html

state_code_lookup = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'}

In [ ]:
state_code_lookup = pd.DataFrame(state_code_lookup, index = [0]).T.reset_index().rename(columns = {'index':'State', 0:'StateAbbr'})
state_code_lookup['State'] = state_code_lookup.State.str.upper()
# state_code_lookup.merge(nass_df_big, how = 'outer')

In [ ]:
nass_df_big = nass_df_big.merge(state_code_lookup).drop(columns = ['State']).rename(columns = {'StateAbbr':'State'})

In [ ]:
nass_df_big

,Year,AgDistrict,County,GRN_BUpACRE,STATE_GRN_BUpACRE,STATE_SLG_TONSpACRE,State
0,1910,CENTRAL,BUFFALO,24.0,26.0,NaN,NE
1,1910,CENTRAL,CUSTER,21.4,26.0,NaN,NE
2,1910,CENTRAL,DAWSON,17.1,26.0,NaN,NE
3,1910,CENTRAL,GREELEY,24.0,26.0,NaN,NE
4,1910,CENTRAL,HALL,31.1,26.0,NaN,NE
...,...,...,...,...,...,...,...
194120,2009,CENTRAL,OTHER (COMBINED) COUNTIES,120.0,150.0,23.0,UT
194121,2009,CENTRAL,UTAH,135.9,150.0,23.0,UT
194122,2009,NORTHERN,BOX ELDER,165.3,150.0,23.0,UT
194123,2009,NORTHERN,OTHER (COMBINED) COUNTIES,169.4,150.0,23.0,UT


## Test to see if there are missing counties

In [ ]:
# check for counties in dat not found in nass_df_big (way 1)

In [ ]:
tmp1 = nass_df_big.loc[:, ['State', 'County']].drop_duplicates()
tmp1['nass'] = True

In [ ]:
tmp2 = dat.loc[:, ['Env', 'County', 'State']].drop_duplicates()
tmp2['dat'] = True

In [ ]:
tmp = tmp1.merge(tmp2, how = 'outer')
tmp

,State,County,nass,Env,dat
0,NE,BUFFALO,True,NaN,NaN
1,NE,CUSTER,True,NaN,NaN
2,NE,DAWSON,True,NaN,NaN
3,NE,GREELEY,True,NaN,NaN
4,NE,HALL,True,NaN,NaN
...,...,...,...,...,...
3084,UT,WASHINGTON,True,NaN,NaN
3085,UT,WAYNE,True,NaN,NaN
3086,UT,SUMMIT,True,NaN,NaN
3087,UT,KANE,True,NaN,NaN


In [ ]:
tmp.loc[((tmp.dat == True) & (tmp.nass == False)), ]

,State,County,nass,Env,dat


In [ ]:
# check for counties in dat not found in nass_df_big (way 2)

for ith_state in list(set(dat.State)):
    counties_found = list(set(nass_df_big.loc[(nass_df_big.State == ith_state), 'County']))
    missings = [e for e in list(set(dat.loc[(dat.State == ith_state), 'County'])) if e not in counties_found]
    if missings != []:
        print(ith_state)
        print(counties_found)
        print(missings)
        print('\n')

In [ ]:

# filter nass_df_big down to counties in the dat dataset
nass_df_big = dat.loc[:, ['State', 'County']].drop_duplicates().merge(nass_df_big)

## Impute mising values

In [ ]:
from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import OneHotEncoder

In [ ]:
# I'd like to use a nested mixed effects model here but geting that running with pymc3 or statsmodels is going to be challenging in the time frame
def iter_imp_nass_df(nass_df_big):
    cat_cols = ['State', 'AgDistrict', 'County']
    float_cols = [e for e in list(nass_df_big) if e not in cat_cols]

    nass_df_big_cats = nass_df_big.loc[:, cat_cols]

    enc = OneHotEncoder()
    enc.fit(nass_df_big_cats)
    nass_onehot = enc.transform(nass_df_big_cats).toarray()
    
    nass_np_cat = np.concatenate([
        nass_onehot,
        np.asanyarray(nass_df_big.drop(columns = cat_cols))
        ], axis = 1)

    imp_mean = IterativeImputer(random_state=0)
    imp_mean.fit(nass_np_cat)

    nass_np_cat = imp_mean.transform(nass_np_cat)

    nass_imp = pd.concat([pd.DataFrame(
        enc.inverse_transform(nass_np_cat[:, 0:nass_onehot.shape[1]]),
        columns = cat_cols
    ), pd.DataFrame(
        nass_np_cat[:, nass_onehot.shape[1]:],
        columns = float_cols
    )
    ], axis = 1)

    return(nass_imp)

In [ ]:
# Impute using each state's data. 
# this is a compromise to get some nesting without spending the time to get 
# a mixed model trained.

nass_df_imp = pd.concat([
    iter_imp_nass_df(nass_df_big = nass_df_big.loc[(nass_df_big.State == e), :])
    for e in 
    tqdm.tqdm(list(set(nass_df_big.State)))
])

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19/19 [00:00<00:00, 58.04it/s]


In [ ]:
nass_df_imp

,State,AgDistrict,County,Year,GRN_BUpACRE,STATE_GRN_BUpACRE,STATE_SLG_TONSpACRE
0,WI,SOUTH CENTRAL,DANE,1928.0,40.000000,37.500000,7.80000
1,WI,SOUTH CENTRAL,DANE,1929.0,36.000000,36.500000,7.80000
2,WI,SOUTH CENTRAL,DANE,1930.0,37.000000,35.500000,7.00000
3,WI,SOUTH CENTRAL,DANE,1931.0,30.000000,29.000000,6.50000
4,WI,SOUTH CENTRAL,DANE,1932.0,41.000000,38.000000,7.50000
...,...,...,...,...,...,...,...
480,IA,CENTRAL,STORY,2018.0,192.300000,201.200000,20.50000
481,IA,CENTRAL,STORY,2019.0,189.300000,193.666667,22.00000
482,IA,CENTRAL,STORY,2020.0,148.400000,188.000000,20.50000
483,IA,CENTRAL,STORY,2021.0,202.100000,199.000000,21.00000


# Merge imputed yield data with G2F data

##  Merge

In [ ]:
dat.head(3)

,Env,Year,Hybrid,Yield_Mg_ha,County,State
0,DEH1_2014,2014,M0088/LH185,5.721725,SUSSEX,DE
1,DEH1_2014,2014,M0143/LH185,11.338246,SUSSEX,DE
2,DEH1_2014,2014,M0003/LH185,6.540810,SUSSEX,DE


In [ ]:
nass_df_imp.head(3)

,State,AgDistrict,County,Year,GRN_BUpACRE,STATE_GRN_BUpACRE,STATE_SLG_TONSpACRE
0,WI,SOUTH CENTRAL,DANE,1928.0,40.0,37.5,7.8
1,WI,SOUTH CENTRAL,DANE,1929.0,36.0,36.5,7.8
2,WI,SOUTH CENTRAL,DANE,1930.0,37.0,35.5,7.0


In [ ]:
tmp = dat.merge(
    # Use the state_code_lookup to replace fully written state names with abbrs
    nass_df_imp, 
    how = 'left')

In [ ]:
tmp.loc[tmp.Year == 2022, ].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11555 entries, 122629 to 134183
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Env                  11555 non-null  object 
 1   Year                 11555 non-null  int64  
 2   Hybrid               11555 non-null  object 
 3   Yield_Mg_ha          0 non-null      float64
 4   County               11555 non-null  object 
 5   State                11555 non-null  object 
 6   AgDistrict           11555 non-null  object 
 7   GRN_BUpACRE          11555 non-null  float64
 8   STATE_GRN_BUpACRE    11555 non-null  float64
 9   STATE_SLG_TONSpACRE  11555 non-null  float64
dtypes: float64(4), int64(1), object(5)
memory usage: 993.0+ KB


In [ ]:
# Some entries lack data. This does not apply to any of the 2022 data so I'll 
# drop the sites lacking historical data.
mask = (tmp.GRN_BUpACRE.notna())
tmp = tmp.loc[mask, ]

In [ ]:
# now the only missing values in dat are Yield_Mg_ha
dat = tmp

## Prepare matrices

In [ ]:
dat = dat.reset_index().drop(columns = 'index')

In [ ]:
def prep_train_test_hist(
test_this_year = '2014',
dat = dat
                        ):
    mask_undefined = (dat.Yield_Mg_ha.isna()) # these can be imputed but not evaluated
    mask_test = ((dat.Year == int(test_this_year)) & (~mask_undefined))
    mask_train = ((dat.Year != int(test_this_year)) & (~mask_undefined))
    test_idx = dat.loc[mask_test, ].index
    train_idx = dat.loc[mask_train, ].index

    ## Encode Hybrid Info ========================================================
    res = []
    for e in list(set(dat['Hybrid'])):
        res += e.split('/')
    res = list(set(res))


    hybrid_onehot = dat.loc[:, ['Hybrid']]#.drop_duplicates(
                                         #).reset_index().drop(columns = ['index'])

    hybrid_onehot = hybrid_onehot.Hybrid.str.split('/', expand=True
                                           ).rename(columns = {0:'p0', 1:'p1'})

    enc = OneHotEncoder()
    enc.fit(np.array(res).reshape([len(res), 1]))

    p0 = enc.transform(hybrid_onehot.loc[:, ['p0']]).toarray()
    p1 = enc.transform(hybrid_onehot.loc[:, ['p1']]).toarray()

    hybrid_mat = p0+p1

    ## Encode Location Info ======================================================
    position_onehot = dat.loc[:, ['Env', 'County', 'State']]

    enc = OneHotEncoder()
    enc.fit(position_onehot)
    position_onehot = enc.transform(position_onehot).toarray()

    ## Turn Yields into array ====================================================
    county_state_mat = np.array(dat.loc[:, ['GRN_BUpACRE', 'STATE_GRN_BUpACRE', 
                                      'STATE_SLG_TONSpACRE']])

    ## Merge into a single array =================================================
    xs_mat = np.concatenate([county_state_mat, hybrid_mat, position_onehot], 
                            axis = 1)

    ## Y =========================================================================
    ys_mat = np.array(dat.loc[:, 'Yield_Mg_ha'])
#     ys_mat = ys_mat.reshape(ys_mat.shape[0], 1)

    train_x = xs_mat[train_idx]
    test_x  = xs_mat[test_idx]

    train_y = ys_mat[train_idx]
    test_y  = ys_mat[test_idx]

    # Reshape to rank 1
    # train_y = train_y.reshape([train_y.shape[0], 1])
    # test_y = test_y.reshape([test_y.shape[0], 1])
    return(train_x, train_y, test_x, test_y, xs_mat, ys_mat)

In [ ]:
# Setup ----------------------------------------------------------------------
trial_name = 'hist'

n_trials= 40
n_jobs = 30


def objective(trial): 
    rf_max_depth = trial.suggest_int('rf_max_depth', 2, 100, log=True)
    rf_n_estimators = trial.suggest_int('rf_n_estimators', 20, 100, log=True)
    rf_min_samples_split = trial.suggest_float('rf_min_samples_split', 0.005, 0.5, log=True)
    
    regr = RandomForestRegressor(
        max_depth = rf_max_depth, 
        n_estimators = rf_n_estimators,
        min_samples_split = rf_min_samples_split
        )
    
    rf = regr.fit(train_x, train_y)
    return (mean_squared_error(train_y, rf.predict(train_x), squared=False))


if True == False:
    reset_trial_name = trial_name

    for test_this_year in ['2021', '2020', '2019', '2018', '2017', '2016', '2015', '2014']:
        print("""
    ------------------------------------------
    ------------------"""+test_this_year+"""------------------
        """)    

        trial_name = reset_trial_name
        trial_name = trial_name+test_this_year
        print(test_this_year)
        # Data Prep. -----------------------------------------------------------------
        # Set up train/test indices --------------------------------------------------
        train_x, train_y, test_x, test_y, xs_mat, ys_mat = prep_train_test_hist(
            test_this_year = test_this_year,
            dat = dat)

        # HPS Study ------------------------------------------------------------------
        cache_save_name = cache_path+trial_name+'_hps.pkl'
        if os.path.exists(cache_save_name):
            study = pkl.load(open(cache_save_name, 'rb'))  
        else:
            study = optuna.create_study(direction="minimize")
            study.optimize(objective, n_trials= n_trials, n_jobs = n_jobs)
            # save    
            pkl.dump(study, open(cache_save_name, 'wb'))    

        print("""
    --------------Study Complete--------------
    ------------------------------------------
        """)
        def fit_single_rep(Rep = 1):
            # Fit Best HPS --------------------------------------------------------------- 
            cache_save_name = cache_path+trial_name+'_'+str(Rep)+'_mod.pkl'

            # Load cached model if it exists
            if os.path.exists(cache_save_name):
                rf = pkl.load(open(cache_save_name, 'rb'))  
            else:
                regr = RandomForestRegressor(
                        max_depth = study.best_trial.params['rf_max_depth'], 
                        n_estimators = study.best_trial.params['rf_n_estimators'],
                        min_samples_split = study.best_trial.params['rf_min_samples_split']
                        )
                rf = regr.fit(train_x, train_y)
                # save    
                pkl.dump(rf, open(cache_save_name, 'wb'))   

            # Record Predictions -----------------------------------------------------
            out = dat.copy()
            out['YHat'] = rf.predict(xs_mat)
            out['YMat'] = ys_mat
            out['Y_center'] = None
            out['Y_scale'] = None
            out['Class'] = trial_name
            out['CV'] = test_this_year
            out['Rep'] = Rep
            out.to_csv('./notebook_artifacts/9_historical/'+trial_name+'_'+str(Rep)+'YHats.csv')
    #         out.to_csv('./data/Shared_Model_Output/'+trial_name+'_'+str(Rep)+'rfYHats.csv')

        # use joblib to get replicate models all at once
        Parallel(n_jobs=10)(delayed(fit_single_rep)(Rep = i) for i in range(10)) #FIXME

In [ ]:
if True == False:
    import os, re
    import shutil

    start_path = cache_path
    end_path   = './data/Shared_Model_Output/'

    yHat_files = [e for e in os.listdir(start_path) if re.match('^hist\d\d\d\d_\dYHats.csv$', e)]

    for yHat_file in yHat_files:
        shutil.move(
            start_path+yHat_file,
            end_path+yHat_file
        )

In [ ]:
if True == False:
    dat.to_csv('./data/Shared_Model_Output/hist9.csv')